# PRUEBAS INICIALES
Para ver la eficacia del modelo pre-entrenado, sin haber hecho ningún ajuste sobre los datos particulares. Registramos los resultados en MLflow

In [1]:
import os
from datasets import load_dataset
import torch
from  metrics.evaluar_metricas import evaluar_metricas_QA, evaluar_metricas_NER
import mlflow
import pprint
import requests
from globals import DATA_DIR,MODELS_DIR

In [2]:
# Realizaremos las pruebas iniciales con el conjunto de test, cuando se haya concluido el entrenamiento del modelo ajustado (usando train y validation) volveremos a tomar la medida con este conjunto de test. Esto nos servirá para ver si efectivamente el modelo ajustado ha mejorado en algún modo el rendimiento del modelo pre-entrenado.
# Datos y modelo
dataset_QA = load_dataset(os.path.join(DATA_DIR, "Escrituras"),'QA',trust_remote_code=True,split='test')
dataset_NER = load_dataset(os.path.join(DATA_DIR, "Escrituras"),'NER',trust_remote_code=True,split='test')

In [3]:
ruta_modelo_QA = os.path.join(MODELS_DIR,'PlanTL-GOB-ES','roberta-large-bne-sqac')
ruta_modelo_NER = os.path.join(MODELS_DIR,'PlanTL-GOB-ES','roberta-base-bne-capitel-ner-plus')
# Tamaño de lote para la inferenca
batch_size = 512

In [4]:
SERVIDOR_MLFLOW = 'http://localhost:5000'
# Debo comprobar si está ejecutando el servidor MLflow, en otro caso se demora la ejecución y acaba dando un error
def mlflow_en_ejecucion(url):
    try:
        response = requests.get(url)        
        # Si el servidor está en ejecución, deberíamos recibir un código de estado HTTP 200
        return response.status_code == 200
    except requests.exceptions.ConnectionError:
        # Si no se puede establecer una conexión, asumimos que el servidor no está en ejecución
        return False
    
assert mlflow_en_ejecucion(SERVIDOR_MLFLOW), f"El servidor MLflow ({SERVIDOR_MLFLOW}) no está en ejecución. Lance 'mlflow ui' desde el terminal."

In [5]:
# Servidor de seguimiento
mlflow.set_tracking_uri(SERVIDOR_MLFLOW)
mlflow.autolog()
mlflow.set_experiment("01 Valoración inicial")
# Impresión elegante de datos en la terminal
pp = pprint.PrettyPrinter(width=150)

2024/02/18 13:13:36 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/02/18 13:13:36 INFO mlflow.tracking.fluent: Autologging successfully enabled for transformers.


In [6]:
# Un vistazo a los datos que estamos testeando
pp.pprint(dataset_QA.info)

DatasetInfo(description='',
            citation='',
            homepage='',
            license='',
            features={'answers': Sequence(feature={'answer_start': Value(dtype='int32', id=None), 'text': Value(dtype='string', id=None)},
                                          length=-1,
                                          id=None),
                      'context': Value(dtype='string', id=None),
                      'id': Value(dtype='string', id=None),
                      'question': Value(dtype='string', id=None)},
            post_processed=None,
            supervised_keys=None,
            task_templates=None,
            builder_name='escrituras',
            dataset_name='escrituras',
            config_name='QA',
            version=0.0.0,
            splits={'test': SplitInfo(name='test', num_bytes=11002385, num_examples=7532, shard_lengths=None, dataset_name='escrituras'),
                    'train': SplitInfo(name='train', num_bytes=35013784, num_examples=240

In [7]:
pp.pprint(dataset_NER.info)

DatasetInfo(description='Dataset para entrenamiento de modelos NER en extracción de datos de escrituras.\n'
                        '                                    Las etiquetas utilizadas se corresponden con los siguientes elementos:\n'
                        "                                    'B-PROTO','I-PROTO':    Número de PROTOCOLO.\n"
                        "                                    'B-FDOC','I-FDOC':      FECHA de firma del DOCUMENTO.\n"
                        "                                    'B-NOT','I-NOT':        NOTARIO, nombre y apellidos.\n"
                        "                                    'B-TDOC','I-TDOC':      TIPO de DOCUMENTO.\n"
                        '                                ',
            citation='',
            homepage='',
            license='',
            features={'id': Value(dtype='string', id=None),
                      'ner_tags': Sequence(feature=ClassLabel(names=['O',
                                      

In [8]:
# Comprobación de si está bien instalado torch y tenemos una GPU
print(torch.cuda.is_available())

True


También he visto que hay varias maneras de realizar las inferencias o consultas sobre el modelo, usando los pipelines. Quiero ver si hay alguna diferencia en el rendimiento. Probaré con distintas formas y cambiaré el tamaño del lote para ver si consigo un rendimiento óptimo.

### Pipeline sobre las características del conjunto de datos
Tras varias pruebas, no se observa una diferencia significativa en el rendimiento con distintos métodos de utilización de los pipelines, el más sencillo y que aprovecha la paralelización y trabajo en lotes, es pasar directamente las features de nuestro dataset al pipeline. 

Este es por tanto el méotodo finalmente implementado en evaluar_metricas_QA, que usaré tanto aquí como al final del entrenamiento QA.

In [10]:
with mlflow.start_run(run_name="Prueba inicial de evaluación QA",description="Predicciones con el modelo pre-entrenado, usando dataset de test"):
    evaluar_metricas_QA(ruta_modelo_QA,dataset_QA,split='test')

..\Models\PlanTL-GOB-ES\roberta-large-bne-sqac
	f1: 0.41752380975914744
	exact: 0.3413436006372809


In [9]:
with mlflow.start_run(run_name="Prueba inicial de evaluación NER",description="Predicciones con el modelo pre-entrenado, usando dataset de test"):
    evaluar_metricas_NER(ruta_modelo_NER,dataset_NER,split='test')

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at ..\Models\PlanTL-GOB-ES\roberta-base-bne-capitel-ner-plus and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([17, 768]) in the checkpoint and torch.Size([9, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([17]) in the checkpoint and torch.Size([9]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


..\Models\PlanTL-GOB-ES\roberta-base-bne-capitel-ner-plus
	f1: 8.413967185527977e-05
	precision: 4.408782294330306e-05
	recall: 0.0009191176470588235
	accuracy: 0.1829839129109202
